In [36]:
from sklearn.datasets import load_files
from nltk.tokenize import TreebankWordTokenizer
import tensorflow
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import re, os, string, random
import numpy as np
from nltk.tokenize import sent_tokenize


https://www.kaggle.com/datasets/shivamkushwaha/bbc-full-text-document-classification
의 bbc text data 를 훈련 데이터 셋으로 사용

전처리 작업을 통해 train data를 list 의 형태로 변환

In [16]:
devided_text =[]

bbc_dataset = os.listdir('/home/hyunhonoh/project/ringle_assignment/bbc')

for i in range(len(bbc_dataset)):
    bbc_text = os.listdir('/home/hyunhonoh/project/ringle_assignment/bbc/' + bbc_dataset[i])
    for j in range(len(bbc_text)):
        f = open('/home/hyunhonoh/project/ringle_assignment/bbc/' + bbc_dataset[i] + '/'+ bbc_text[j], encoding='latin1')
        F = f.readlines()
        devided_text = devided_text + F

while '\n' in devided_text:
    devided_text.remove('\n')

print(len(devided_text))





12893


In [17]:
train_data = devided_text[:3000]
test_data = devided_text[3000:3500]



전처리 과정: 
 모두 소문자 / (숫자 제거, 불용어 제거 취소) / 공백 제거 / Treebank tokenize

In [18]:
tokenizer = TreebankWordTokenizer()
train_data_sent = []
shortword = re.compile(r'\W*\b\w{1,2}\b')


# 여기까지 해서 모든 데이터를 한 문장씩 string 의 형태로 list 에 저장함
for i in range(len(train_data)):
    train_data_sent += sent_tokenize(train_data[i])
print(type(train_data_sent[1]))

print(train_data_sent[1])


for i in range(len(train_data_sent)):
    train_data_sent[i] = shortword.sub('', train_data_sent[i])
    train_data_sent[i] = train_data_sent[i].lower()


print(train_data_sent[1])
# print(len(train_data_sent))




<class 'str'>
General Motors (GM) saw its net profits fall 37% in the last quarter of 2004, as it continued to be hit by losses at its European operations.
general motors) saw its net profits fall the last quarter 2004 continued hit losses its european operations.


1. Compound sentence refers to a sentence made up of two independent clauses connected to one another with a coordinating conjunction, such as FANBOYS (for , and, nor, but, or, yet, so). 
-> COMPOUND 는 두 독립적인 문장이 합쳐져서 만들어진 문장으로, 접속사로 fanboys를 사용한다.

2. COMPLEX SENTENCE is made up of an independent clause and one or more dependent clauses connected to it. Dependent clauses begin with subordinating conjunctions, such as (after, although, as, because, ...)

앞서 nltk를 이용해서 문장 단위로 분리해둔 단위 문장들을 spacy 의 postagging 기능을 이용해서 분석

In [20]:
import en_core_web_sm, en_core_web_trf
nlp = en_core_web_sm.load()
tokenized_list=[]
doc =[]

for i in range(len(train_data_sent)):
    doc.append(nlp(train_data_sent[i]))
    tokenized = [(token.text, token.dep_) for token in doc[i]]
    tokenized_list.append(tokenized)





In [23]:
import collections

POS_list=[] #POS list는 각 문장의 POS 값들을 모아놓은 리스트


for j in range(len(tokenized_list)):
    POS=[]
    for i in range(len(tokenized_list[j])):
        POS.append(tokenized_list[j][i][1])
    POS_list.append(POS)

#실제 문장의 수와 POS 모음이 일치하는지 확인
print('POS_list의 인덱스값 = {}'.format(len(POS_list)))
print('tokenized_list의 인덱스 값 = {}'.format(len(tokenized_list)))


POS_list의 인덱스값 = 9537
tokenized_list의 인덱스 값 = 9537


구글링을 통해 스스로 판단한 simple, complex, compound의 정의를 생각해서 아래와 같이 스스로 데이터를 분류해봄

In [24]:
simple_sent=[]
complex_sent=[]
compound_sent=[]
compl_compo_sent=[]
for i in range(len(POS_list)):
    if  POS_list[i].count('aux') + POS_list[i].count('ROOT') ==1 \
        and POS_list[i].count('cc') ==0 \
        and POS_list[i].count('prep') == 0 \
        and POS_list[i].count('advmod') ==0 \
        and POS_list[i].count('mark') == 0 \
        and POS_list[i].count('punct') == 1 \
        and POS_list[i].count('acl')==0\
        and POS_list[i].count('advcl')==0:
        simple_sent.append(train_data_sent[i])
    elif POS_list[i].count('cc') > 0 \
        and POS_list[i].count('aux') ==2 \
        and POS_list[i].count('nsubj')==2:
        compound_sent.append(train_data_sent[i])
    elif (POS_list[i].count('mark') == 1 \
        or POS_list[i].count('advmod') ==1 \
        or POS_list[i].count('prep') ==1)\
        and POS_list[i].count('punct') < 3:
        complex_sent.append(train_data_sent[i])
    else :
        compl_compo_sent.append(train_data_sent[i])



print('number of simple_sent = {}'.format(len(simple_sent)))
print('number of compound_sent = {}'.format(len(compound_sent)))
print('number of complex_sent = {}'.format(len(complex_sent)))
print('number of compl_compo_sent = {}'.format(len(compl_compo_sent)))

for i in range(5):
    print(simple_sent[i])



    


number of simple_sent = 305
number of compound_sent = 165
number of complex_sent = 3470
number of compl_compo_sent = 5597
the trade deficit large part the latter.
that pushed the month deficit $500.5bn.
low-cost airlines hit eurotunnel
 shareholder revolt threw out the old board 2004.
their economic targets became known the lisbon agenda.


train data set 자체의 수가 매우 부족한 상황이지만, 기준을 낮게 잡아서 데이터의 수를 무작정 늘리는 것보다는 train data set 이 깔끔한게 더 좋을 것으로 판단하였다.
동일한 학습 환경을 조성하는 것이 옳다고 생각해서 우선 모든 train data set을 160개로 고정하도록 한다.

In [25]:
simple_trainset = []
compound_trainset = []
complex_trainset = []
compl_compo_trainset = []

for i in range(160):
    simple_trainset.append(simple_sent[i])
    compound_trainset.append(compound_sent[i])
    complex_trainset.append(complex_sent[i])
    compl_compo_trainset.append(compl_compo_sent[i])

미처 안한 불용어 제거와 punctuation 제거, 숫자 제거

In [27]:

prepro_simple_trainset =[]
prepro_complex_trainset =[]
prepro_compound_trainset =[]
prepro_compl_compo_trainset =[]
translator = str.maketrans('', '', string.punctuation)
stop_words = set(stopwords.words('english'))


for sentence in simple_trainset:
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)    
    sentence = sentence.translate(translator)
    token = tokenizer.tokenize(sentence)
    result=[]
    for word in token:
        if word not in stop_words:
            if len(word):
                result.append(word)
    prepro_simple_trainset.append(result)
for sentence in complex_trainset:
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)    
    sentence = sentence.translate(translator)
    token = tokenizer.tokenize(sentence)
    result=[]
    for word in token:
        if word not in stop_words:
            if len(word):
                result.append(word)
    prepro_complex_trainset.append(result)
for sentence in compound_trainset:
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    sentence = sentence.translate(translator)
    token = tokenizer.tokenize(sentence)
    result=[]
    for word in token:
        if word not in stop_words:
            if len(word):
                result.append(word)
    prepro_compound_trainset.append(result)
for sentence in compl_compo_trainset:
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    sentence = sentence.translate(translator)
    token = tokenizer.tokenize(sentence)
    result=[]
    for word in token:
        if word not in stop_words:
            if len(word):
                result.append(word)
    prepro_compl_compo_trainset.append(result)



nltk 의 tokenizer를 이용해서 fit_on_text를 하여 각 문장에 전처리 후에 남은 분석에 사용하기가 용이안 단어들이 각각 얼마나 남았는지 판단
texts_to_sequence를 통해 정수 인코딩 진행

In [28]:
tokenizer_encode = Tokenizer()
tokenizer_encode.fit_on_texts(prepro_simple_trainset + prepro_complex_trainset \
    + prepro_compound_trainset + prepro_compl_compo_trainset)

encoded_simple_trainset = tokenizer_encode.texts_to_sequences(prepro_simple_trainset)
encoded_compound_trainset = tokenizer_encode.texts_to_sequences(prepro_compound_trainset)
encoded_complex_trainset = tokenizer_encode.texts_to_sequences(prepro_complex_trainset)
encoded_compl_compo_trainset = tokenizer_encode.texts_to_sequences(prepro_compl_compo_trainset)


IMDB movie review dataset 과 유사하게 만들어주기 위해서 ndarray 형태로 만들어주고 이를 셔플해서 섞어줌.

In [30]:
tot_trainset=[encoded_simple_trainset, encoded_compound_trainset, \
    encoded_complex_trainset, encoded_compl_compo_trainset]
index=0
data=[]
label=[]
for trainset in tot_trainset:
    data.append(np.array([trainset[i] for i in range(160)], dtype=object))
    label.append(np.full(160, index, dtype=np.int32))
    index +=1

total_data = np.concatenate(data, 0)
total_label = np.concatenate(label, 0)

# shuffle data

shuffle_case=np.arange(total_data.shape[0])
np.random.shuffle(shuffle_case)

shuffle_data = total_data[shuffle_case]
shuffle_label = total_label[shuffle_case]
shuffle_data[3]


[2596, 2597, 2598, 2599, 2600, 1040, 622, 131, 1044]

문장의 최대 길이를 확인하여, 이에 맞게 padding을 씌워주도록 하여 데이터셋의 크기를 일정하게 만들어준다.

In [32]:
data_length = [len(data) for data in shuffle_data]

print('문장의 최대 길이 : {}'.format(np.max(data_length)))
print('문장의 평균 길이 : {}'.format(np.mean(data_length)))


문장의 최대 길이 : 26
문장의 평균 길이 : 10.140625


모델은 1D convolution 모델을 사용하였다.
1. 모델의 전반적인 구조는 '딥러닝을 이용한 자연어처리 입문'을 참고하였다.

2. 임베딩 레이어에서 정수 인코딩 + 패딩이 씌워진 데이터들을 임베딩 벡터의 형태로 만들어주는데, 이때 벡터의 차원 수는 256 으로 하였다.

3. convolution layer는 학습의 정확도가 크게 높아지지 않아서 parameter의 수를 늘리기 위해 기존에 하나 있던 것을 두개로 늘렸는데, 큰 소용이 없었다.

4. convolution layer 의 뉴런의 수는 256개로 설정하였고, 마지막의 denselayer 를 거친 후의 차원 수는 validation 에 용이하게 만들기 위해서 낮추었는데, 
한번에 많이 낮추게 되면 왠지 정보 손실이 일어날 것 같아서 두번 거쳤다.

5. drop out layer를 중간에 넣어서 과도하게 튀는 값들을 잡고자 하였다.

In [33]:
import re
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dropout, GlobalMaxPooling1D, Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [47]:
max_len=26
vocab_size = 3000
embedding_dim = 256
padded = pad_sequences(shuffle_data, maxlen=max_len)
X_train_set =padded[:540]
X_test_set = padded[540:]
Y_train_set =shuffle_label[:540]
Y_test_set =shuffle_label[540:]

print(X_test_set.shape)

(100, 26)


모델의 정확도는 크게 좋지 않았는데, 몇가지 이유에 대해서 생각해보자면,

1. 너무 과도한 기준의 설정으로 인해 데이터셋의 절대적인 양이 줄어버렸다. - 데이터셋의 크기와 질은 딥러닝에서 가장 중요하다고 생각하므로, 아마 가장 큰 요인이 아닐까 싶다.

2. 목적에 맞지 않는 전처리 - nlp 에서 sentiment analysis 등의 분야에서 흔하게 사용되는 전처리 과정을 모두 사용해보고자 하였는데, 문장 구조에 따라서 simple 인지 complex인지 compound 인지 구분하는 task 였기 때문에
    필요하지 않은 전처리 과정을 거치면서 문장에 중요한 정보들이 많이 사라졌을 수 있겠다고 생각하였다.

3. 모델의 부적합 - image data 에 대한 classifying을 더 많이 해보아서 더 익숙한 convolution model로 과제를 진행하였는데, 텍스트 데이터에 한해서는 RNN 기반 모델이 더 좋은 성능을 낼 수 있다는 생각을 하였다.

In [48]:

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(0.3))
model.add(Conv1D(filters=256, kernel_size=3, padding='valid', activation='relu'))
model.add(Conv1D(filters=256, kernel_size=3, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('best_model2.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train_set, Y_train_set, epochs=20, validation_data=(X_test_set, Y_test_set), callbacks=[es, mc])

Epoch 1/20
16/17 [===========================>..] - ETA: 0s - loss: -2.7773 - acc: 0.1953
Epoch 1: val_acc improved from -inf to 0.27000, saving model to best_model2.h5
17/17 [==============================] - 1s 33ms/step - loss: -2.8310 - acc: 0.2000 - val_loss: -6.8617 - val_acc: 0.2700
Epoch 2/20
15/17 [=========================>....] - ETA: 0s - loss: -7.5245 - acc: 0.2729
Epoch 2: val_acc did not improve from 0.27000
17/17 [==============================] - 0s 22ms/step - loss: -7.7615 - acc: 0.2648 - val_loss: -6.8622 - val_acc: 0.2400
Epoch 3/20
16/17 [===========================>..] - ETA: 0s - loss: -7.7735 - acc: 0.2266
Epoch 3: val_acc did not improve from 0.27000
17/17 [==============================] - 0s 22ms/step - loss: -7.7658 - acc: 0.2296 - val_loss: -6.8622 - val_acc: 0.2400
Epoch 4/20
16/17 [===========================>..] - ETA: 0s - loss: -7.9224 - acc: 0.2734
Epoch 4: val_acc did not improve from 0.27000
17/17 [==============================] - 0s 22ms/step - l

급하게 진행하느라 더 깊이 공부하고 과제를 진행하지 못한 점 죄송합니다. 

감사합니다! - 노현호 올림.